# 修了課題④　CIFAR-10

**データセット**：CIFAR-10データセット（Canadian Institute For Advanced Research）（以下、CIFAR-10）は、

- ラベル「0」：airplane（飛行機）
- ラベル「1」：automobible（自動車）
- ラベル「2」：bird（鳥）
- ラベル「3」：cat（猫）
- ラベル「4」：deer（鹿）
- ラベル「5」：dog（犬）
- ラベル「6」：frog（カエル）
- ラベル「7」：horse（馬）
- ラベル「8」：ship（船）
- ラベル「9」：truck（トラック）

という10種類の「物体カラー写真」（乗り物や動物など）の画像データセットである。

CIFAR-10は、主に画像認識を目的としたディープラーニング／機械学習の研究や初心者向けチュートリアルで使われている。CIFAR-10は上記の通り10クラス（種類）となっており手軽に扱えるが、より複雑な内容として100クラス版であるCIFAR-100も提供されている。

CIFAR-10データセット全体は、

- 5万枚の訓練データ用（画像とラベル）
- 1万枚のテストデータ用（画像とラベル）
- 合計6万枚

で構成される（※「ラベル」= 正解を示す教師データ）。また各画像のフォーマットは、

- 24bit RGBフルカラー画像：RGB（赤色／緑色／青色）3色の組み合わせで、それぞれ「0」〜「255」の256段階
- 幅32x高さ32ピクセル：1つ分のデータが基本的に(3,32,32)もしくは(32,32,3)（=計3072画素）という多次元配列の形状となっており、最初もしくは最後の次元にある3要素がRGB値

となっている（※「ピクセル」=画素のこと。RGB形式であるため、簡単に画像化できる）。

**合格基準**：正解率 85%以上です

##作成までの流れ
大まかな流れとして
1. データのダウンロードと正規化  
   torchvisionというライブラリを使用して、CIFAR10の訓練用のデータ、テスト用のデータをダウンロードします。  
   また、ダウンロードした画像に対して正規化を行います。

2. モデルの構築  
   学習を行うモデルの各層の役割を理解して、構築します。

3. 損失関数などの設定  
   学習を行うのに必要な損失関数などの設定を行います。

4. 学習と結果  
   訓練データで学習を行い、どのくらいの精度があるのかを、テスト用データを使って確認します。

##必要なライブラリーのインポートとGoogleDriveへの接続

In [ ]:
#GoogleDriveへの接続を行う
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#必要なライブラリーのインポート
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#ダウンロードに必要なライブラリーのインポート
import pickle
from PIL import Image
import os

#1.データのダウンロードと正規化

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=15kspx4XmoR5Kh1fKkdxjjPcn_Y8tkaP3' -O train.pickle

--2025-01-29 06:22:09--  https://drive.google.com/uc?export=download&id=15kspx4XmoR5Kh1fKkdxjjPcn_Y8tkaP3
Resolving drive.google.com (drive.google.com)... 74.125.142.101, 74.125.142.138, 74.125.142.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=15kspx4XmoR5Kh1fKkdxjjPcn_Y8tkaP3&export=download [following]
--2025-01-29 06:22:09--  https://drive.usercontent.google.com/download?id=15kspx4XmoR5Kh1fKkdxjjPcn_Y8tkaP3&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.142.132, 2607:f8b0:400e:c0d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.142.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31319904 (30M) [application/octet-stream]
Saving to: ‘train.pickle’

train.pickle        100%[===================>]  29.87M  76.4MB/s  

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1-QKklgEpROkVIUnaLQ9dKgfCK_mp78xN' -O val.pickle

--2025-01-29 06:22:14--  https://drive.google.com/uc?export=download&id=1-QKklgEpROkVIUnaLQ9dKgfCK_mp78xN
Resolving drive.google.com (drive.google.com)... 74.125.142.101, 74.125.142.138, 74.125.142.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1-QKklgEpROkVIUnaLQ9dKgfCK_mp78xN&export=download [following]
--2025-01-29 06:22:14--  https://drive.usercontent.google.com/download?id=1-QKklgEpROkVIUnaLQ9dKgfCK_mp78xN&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.142.132, 2607:f8b0:400e:c0d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.142.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6264552 (6.0M) [application/octet-stream]
Saving to: ‘val.pickle’

val.pickle          100%[===================>]   5.97M  --.-KB/s    

In [ ]:
# バイナリファイルを読み込んでから、画像データに変換処理を行う。
def parse_pickle(rawdata, dataset_name):
    for i in range(10):
        dir = dataset_name + "/" + f"{i:02d}"
        if not os.path.exists(dir):
            os.makedirs(dir)
    m = len(rawdata["data"])
    for i in range(m):
        filename = f'{i}.png'
        label = rawdata["label"][i]
        data = rawdata["data"][i]
        data = data.reshape(3, 32, 32)
        data = np.swapaxes(data, 0, 2)
        data = np.swapaxes(data, 0, 1)
        with Image.fromarray(data) as img:
            img.save(f"{dataset_name}/{label:02d}/{filename}")

train = {'label':[], 'data':[]}
with open('train.pickle', "rb") as fp:
  train = pickle.load(fp, encoding="latin-1")
parse_pickle(train, "train")

with open('val.pickle', "rb") as fp:
  val = pickle.load(fp, encoding="latin-1")
parse_pickle(val, "val")

In [ ]:
# データ拡張の設定
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                #もっと増やしてみてもいいかもしれません
                                ])

In [ ]:
# バッチサイズの設定
batch_size = 25

# データローダーの設定
trainset = torchvision.datasets.ImageFolder(root='train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

valset = torchvision.datasets.ImageFolder(root='val', transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print('学習データ:', len(trainset))
print('検証データ:', len(valset))

学習データ: 10000
検証データ: 2000


In [ ]:
# pytorch のライブラリーを利用して、事前学習の重みをロード済みのモデルインスタンスを作成する。
# なお、pretrained=True とすると事前学習モデルとなり、Falseとするとモデルのみが作成される。
net = torchvision.models.convnext_base(pretrained=True)
net

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Base_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Base_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/convnext_base-6075fbad.pth" to /root/.cache/torch/hub/checkpoints/convnext_base-6075fbad.pth
100%|██████████| 338M/338M [00:04<00:00, 77.9MB/s]


ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=128, out_features=512, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=512, out_features=128, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(

In [ ]:
# 分類器部分を cifar10 用に付け替える。
net.classifier[2] = nn.Linear(1024 ,out_features=10)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
net = net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from tqdm import tqdm

# 学習エポックの設定
epoch_num = 50

# 学習ループの設定
for epoch in tqdm(range(epoch_num)):  # エポックの進行度を表示するためにtqdmを使用

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): # leave=Falseで内部のプログレスバーが完了後に消えるように設定
        inputs, labels = data
        optimizer.zero_grad()

        # テンソルをGPUに移動
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 結果表示
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')


100%|██████████| 50/50 [22:16<00:00, 26.72s/it]

Finished Training


##結果のモデルを保存する


In [ ]:
vgg_pre_weight_path = './vgg_pre_weight_path.pth'
torch.save(net.state_dict(), vgg_pre_weight_path)

##結果を検証用データで確認する

In [ ]:
net.load_state_dict(torch.load(vgg_pre_weight_path))

<ipython-input-15-3d57d66b2b7d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(vgg_pre_weight_path))


<All keys matched successfully>

In [ ]:
correct = 0
total = 0
# 勾配を記憶せず（学習せずに）に計算を行う
with torch.no_grad():
    for data in valloader:
        images, labels = data

        images = images.to(device)
        labels = labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('正解率 : %d %%' % (100 * correct / total))

正解率 : 87 %


# 提出形式

## テスト用データセットのダウンロード

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1-T-luRcFf14qV_rR66B3groh8imA-8lo' -O test_data.pickle

--2025-01-29 06:49:45--  https://drive.google.com/uc?export=download&id=1-T-luRcFf14qV_rR66B3groh8imA-8lo
Resolving drive.google.com (drive.google.com)... 142.250.107.113, 142.250.107.100, 142.250.107.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.107.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1-T-luRcFf14qV_rR66B3groh8imA-8lo&export=download [following]
--2025-01-29 06:49:45--  https://drive.usercontent.google.com/download?id=1-T-luRcFf14qV_rR66B3groh8imA-8lo&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.197.132, 2607:f8b0:400e:c08::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.197.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6259881 (6.0M) [application/octet-stream]
Saving to: ‘test_data.pickle’

test_data.pickle    100%[===================>]   5.97M  --

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

with open('test_data.pickle', "rb") as fp:
  test = pickle.load(fp, encoding="latin-1")

for i in range(len(test['data'])):
  data = test["data"][i]
  data = data.reshape(3, 32, 32)
  data = np.swapaxes(data, 0, 2)
  data = np.swapaxes(data, 0, 1)
  img = transform(data)
  img = torch.unsqueeze(img, 0)
  if i==0:
    images=img
  else:
    images = torch.cat([images, img])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.eval()
images = images.to(device)
with torch.no_grad():
    outputs = net(images)
    _, predictions = torch.max(outputs, 1)
print(predictions)

tensor([1, 9, 9,  ..., 2, 8, 0], device='cuda:0')


In [ ]:
# pandasのDataFrame形式に変換し、CSV出力する
import pandas as pd
y_pred = pd.DataFrame(predictions.cpu(), columns=['number'])
y_pred.to_csv('y_pred.csv')
y_pred

,number
0,1
1,9
2,9
3,4
4,2
...,...
1995,7
1996,8
1997,2
1998,8


In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1-UHqW8wgH46J-ltEdUfOX-DounUbZMAI' -O test_label.pickle
with open('test_label.pickle', "rb") as fp:
  test_label = pickle.load(fp, encoding="latin-1")

--2025-01-29 06:50:51--  https://drive.google.com/uc?export=download&id=1-UHqW8wgH46J-ltEdUfOX-DounUbZMAI
Resolving drive.google.com (drive.google.com)... 108.177.98.101, 108.177.98.102, 108.177.98.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.98.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1-UHqW8wgH46J-ltEdUfOX-DounUbZMAI&export=download [following]
--2025-01-29 06:50:51--  https://drive.usercontent.google.com/download?id=1-UHqW8wgH46J-ltEdUfOX-DounUbZMAI&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.197.132, 2607:f8b0:400e:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.197.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4023 (3.9K) [application/octet-stream]
Saving to: ‘test_label.pickle’

test_label.pickle   100%[===================>]   3.93K  --.-KB/s

In [ ]:
labels = torch.tensor(test_label['label'])
correct = (predictions.cpu() == labels).sum().item()
assert len(predictions) == len(labels)
print( f"正解率 : {100 * correct // len(labels)} %" )

正解率 : 87 %
